## recovery 전처리 시작

In [1]:
# !pip install --user python-mecab-ko pyLDAvis

### 쿠팡이랑 11번가 댓글 합치기

In [2]:
import pandas as pd

In [3]:
# df_recover_coup=pd.read_csv('../datasets/recovery_coupang.csv')
# df_recover_11st=pd.read_csv('../datasets/recovery_eleven.csv')

In [4]:
# df_recovery_raw=pd.concat([df_recover_coup, df_recover_11st], axis=0)
# df_recovery_raw

In [5]:
# df_recovery_raw.isnull().sum()

In [6]:
# df_recovery_raw=df_recovery_raw.dropna()

In [7]:
# df_recovery_raw

In [8]:
# df_recovery_raw=df_recovery_raw.drop(columns=['Unnamed: 0','_id'])

In [9]:
# df_recovery = df_recovery_raw.reset_index(drop=True)
# df_recovery

In [10]:
# df_recovery.to_csv('../datasets/recovery.csv')
df_recovery=pd.read_csv('../datasets/recovery.csv')

In [11]:
df_recovery=df_recovery.drop(columns=['Unnamed: 0'])

### 불용어 처리 및 한글자 살리기 및 단어 치환, 

#### 전처리 필요한 파일들 가져오기

In [12]:
# 대체어
df_replace = pd.read_excel('../datasets/sa_replace_list.xlsx')
df_replace[:5]

,before_replacement,after_replacement
0,만졷ㄱ,만족
1,않좋아서,안좋아서
2,괜찮네요,보통이예요
3,시킬껭,시킬게요
4,용,요


In [13]:
# 불용어 
df_stop = pd.read_excel('../datasets/sa_stopword_list.xlsx')
df_stop[:5]

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


In [14]:
# 한글자 살리기
df_oneword=pd.read_excel('../datasets/sa_one_char_list.xlsx')
df_oneword[:5]

,one_char_keyword
0,컵
1,방
2,물
3,돈
4,꿈


#### 전처리 시작 
- 1. 단어 치환 하기 
- 2. 불용어 처리 하기 
- 3. okt 이용해서 stemming 하기 _ 부사를 동사 원형으로 바꿔서 보기
<!-- mecab.pos로 토큰화하기 (.append(' '.join(token)) okt 하기  -->


In [15]:
## 단어 치환 함수 블로그 복사
def replace_word(review):
    for number in range(len(df_replace['before_replacement'])):
        try: 
            # 치환 단어만 데이터 치환
            if replace_list['before_replacement'][number] in review_content :
                review_content = review.replace(replace_list['before_replacement'][number], replace_list['after_replacement'][number])
        except: 
            pass
    return review
                

In [16]:
## 단어 치환 함수 블로그 복사
df_recovery['reveiw_replaced'] = ''
review_replaced_list = list()
for review in df_recovery['review_content']:
    review_replaced = replace_word(str(review))
    review_replaced_list.append(review_replaced)
df_recovery['reveiw_replaced'] = review_replaced_list
df_recovery[-10:]

,product_name,review_date,review_content,review_star,review_writer,reveiw_replaced
21764,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.09.23,배송 빨라요 성능 보통이에요 유통기한 보통이에요\n제품 섭취한지 얼마되지않아.좋은지...,3.0,jong******,배송 빨라요 성능 보통이에요 유통기한 보통이에요\n제품 섭취한지 얼마되지않아.좋은지...
21765,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.03.03,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요\n아침마다 일어나는게 힘...,5.0,daph****,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요\n아침마다 일어나는게 힘...
21766,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.02.27,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요\n요즘 들어 기력이 떨어...,5.0,neti*****************,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요\n요즘 들어 기력이 떨어...
21767,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.02.27,배송 빨라요 성능 성능이 뛰어나요 유통기한 보통이에요\n지금까지 먹어본것중 아르기닌...,5.0,with*******,배송 빨라요 성능 성능이 뛰어나요 유통기한 보통이에요\n지금까지 먹어본것중 아르기닌...
21768,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.08.09,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요,5.0,jhi8**,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요
21769,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.07.15,배송 빨라요 성능 성능이 별로예요 유통기한 기한이 넉넉해요\n2박스 먹어봤으나 피로...,2.0,rkdg****,배송 빨라요 성능 성능이 별로예요 유통기한 기한이 넉넉해요\n2박스 먹어봤으나 피로...
21770,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.07.07,배송 빨라요 성능 성능이 별로예요 유통기한 보통이에요\n호불호가 있을지 모르겠지만 ...,2.0,macg****************,배송 빨라요 성능 성능이 별로예요 유통기한 보통이에요\n호불호가 있을지 모르겠지만 ...
21771,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.06.16,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요,5.0,rkdg****,배송 빨라요 성능 성능이 뛰어나요 유통기한 기한이 넉넉해요
21772,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.04.06,배송 보통이에요 성능 보통이에요 유통기한 보통이에요\n지인 선물을줬는데 좋아했어요,5.0,채슨생,배송 보통이에요 성능 보통이에요 유통기한 보통이에요\n지인 선물을줬는데 좋아했어요
21773,푸드올로지 윤성빈PICK 에너지붐 /고함량 액상 아르기닌/ 피로회복,2022.03.27,배송 보통이에요 성능 성능이 뛰어나요 유통기한 보통이에요,5.0,smcd***,배송 보통이에요 성능 성능이 뛰어나요 유통기한 보통이에요


In [41]:
# 내용의 토큰화 먼저_healthapp 복사
from mecab import MeCab
mecab = MeCab()
from konlpy.tag import Okt
okt = Okt()
stop=df_stop['stopword'].tolist()
oneword=df_oneword['one_char_keyword'].tolist()

In [42]:
# 불용어 제거 _ 블로그 및 chat
def stopword(text) :
    words = text.split() # text를 공백 기준으로 단어 분할한 뒤, 불용어 처리 진행 
    removed_words = [word for word in words if word not in stop]
    return ' '.join(removed_words)

In [45]:
## 불용어 처리 완료 
df_recovery['stopremoved_reviews']=df_recovery['reveiw_replaced'].apply(stopword)

In [44]:
# tokenized_reviews = list()
# # df_recovery['stopremoved_reviews'] 열의 각 텍스트에 대해 토큰화 수행 
# for review_text in df_recovery['stopremoved_reviews']:
#     tokens = []
#     try:
#         for word in mecab.pos(review_text):
#             token, pos = word[0], word[1]
#             if len(token) > 1 or token in oneword:
#                 tokens.append(token)
#     except:
#         pass
#     # tokenized_reviews.append(tokens)
#     tokenized_reviews.append(' '.join(tokens)) # okt 형태소도 분석하기 위해 작업함. 

In [ ]:
tokenized_reviews = list()
# df_recovery['stopremoved_reviews'] 열의 각 텍스트에 대해 토큰화 수행 
for review_text in df_recovery['stopremoved_reviews']:
    tokens = []
    try:
        for word, pos in okt.pos(review_text, norm=True, stem=True):
            if len(word) > 1 or word in oneword:
                tokens.append(word)
    except:
        pass
    tokenized_reviews.append(' '.join(tokens))

In [38]:
# 토큰화된 결과를 새로운 열 'tokenized_review'에 추가
df_recovery['tokenized_review'] = tokenized_reviews

In [40]:
tokenized_review=df_recovery['tokenized_review']
tokenized_review

0                               아요 피곤 느낌 사라졌 네요 물 삼키 고요 ..
1                                복용 한지 얼마 않 지만 피로 회복 활력 증강
2        함량 으면서 품질 제품 구매 어요 항상 체력 부분 굉장히 힘들 어서 한약 면서 체력...
3        업무 강도 져서 인지 입맛 힘들 길래 구매 어요 성분 함량 다른 보다 아서 괜찮 더...
4        코로나 확진 떨어진 컨디션 올라오 않 아서요 블랙 마카 남자 한테 특화 영양제 어서...
                               ...                        
21769    배송 빨라요 성능 성능 별로 예요 유통 기한 기한 넉넉 해요 박스 봤으나 피로 회복...
21770    배송 빨라요 성능 성능 별로 예요 유통 기한 보통 에요 호불호 을지 모르 지만 너무...
21771                      배송 빨라요 성능 성능 뛰어나 유통 기한 기한 넉넉 해요
21772            배송 보통 에요 성능 보통 에요 유통 기한 보통 에요 지인 선물 는데 어요
21773                       배송 보통 에요 성능 성능 뛰어나 유통 기한 보통 에요
Name: tokenized_review, Length: 21774, dtype: object

In [23]:
# ## mecab.nouns()로 진행해보기 → okt로 변경하기
# tokenized_nouns_reviews = list()
# # df_recovery['reveiw_replaced'] 열의 각 텍스트에 대해 토큰화 수행
# for review_text in df_recovery['reveiw_replaced']:
#     tokens = []
#     try:
#          # mecab.nouns() 메소드로 명사 추출
#         nouns = mecab.nouns(review_text)
#         # 추출된 명사 중에서 길이가 1 이상인 명사 또는 one word에 있는 명사만 포함
#         for noun in nouns:
#             if len(noun) > 1 or noun in oneword:
#                 tokens.append(noun)
#     except:
#         pass
#     tokenized_nouns_reviews.append(tokens)